In [3]:
import ipyparallel as parallel

In [6]:
clients = parallel.Client()
clients.block = True
print(clients.ids)

[0, 1]


In [ ]:
dview = clients.direct_view()

%px import numpy as np
%px r = np.random.random(3)
%px print(r)

In [18]:
r = dview.gather("r")
print(r)

[ 0.42946717  0.30908573  0.90743979  0.103505    0.21330654  0.39515152
  0.69420744  0.40598522  0.9369325   0.57544252  0.2404624   0.78730275]


In [24]:
import sympy
pi = sympy.pi.evalf(400)
pi
digits = (d for d in str(pi)[2:])
digits
pi

3.141592653589793238462643383279502884197169399375105820974944592307816406286208998628034825342117067982148086513282306647093844609550582231725359408128481117450284102701938521105559644622948954930381964428810975665933446128475648233786783165271201909145648566923460348610454326648213393607260249141273724587006606315588174881520920962829254091715364367892590360011330530548820466521384146951941511609

In [27]:
digits = (d for d in str(pi)[2:])
freqs = one_digit_freqs(digits)

NameError: name 'one_digit_freqs' is not defined

In [28]:
#For 10000 values
%px import pandas as pd
%px import numpy as np
%px import random
%px import timeit, functools
%px import copy

#%px def random_array():
%px     apple_df = pd.read_csv('data\\apple_2011.csv',names =['date','price','percent_change'],skiprows=1)
%px     random.seed(7340)
%px     last_price = apple_df['price'].loc[apple_df['price'].idxmax()]
%px     final_df = np.array([])
%px 
%px     count=20
%px     final_value = 0
%px     for j in range(0,10000):
%px         sample_df = pd.DataFrame(apple_df.percent_change.sample(n=count))
%px         sample_df = sample_df.reset_index(drop=True)
%px         sample_df['updated_price'] = 0.0
%px         sample_df.set_value(0,'updated_price',last_price + last_price*sample_df['percent_change'].loc[sample_df.index[0]])
%px         sample_df['percent_change']=sample_df['percent_change'] + 1
%px         for i in range(1,count):
%px             sample_df.set_value(i,'updated_price',sample_df.updated_price.iloc[[i-1]] * sample_df['percent_change'].loc[sample_df.index[i]])
%px         final_value = sample_df.get_value(count-1,'updated_price')
%px         final_df = np.append(final_df,final_value)


CompositeError: one or more exceptions from call to method: execute
[0:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist
[1:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist
[2:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist
[3:execute]: FileNotFoundError: File b'data\\apple_2011.csv' does not exist

In [32]:
"""Example showing how to merge multiple remote data streams.
"""
# Slightly modified version of:
# http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/511509

import heapq
from IPython.parallel.error import RemoteError

def mergesort(list_of_lists, key=None):
    """ Perform an N-way merge operation on sorted lists.

    @param list_of_lists: (really iterable of iterable) of sorted elements
    (either by naturally or by C{key})
    @param key: specify sort key function (like C{sort()}, C{sorted()})

    Yields tuples of the form C{(item, iterator)}, where the iterator is the
    built-in list iterator or something you pass in, if you pre-generate the
    iterators.

    This is a stable merge; complexity O(N lg N)

    Examples::

    >>> print list(mergesort([[1,2,3,4],
    ...                      [2,3.25,3.75,4.5,6,7],
    ...                      [2.625,3.625,6.625,9]]))
    [1, 2, 2, 2.625, 3, 3.25, 3.625, 3.75, 4, 4.5, 6, 6.625, 7, 9]

    # note stability
    >>> print list(mergesort([[1,2,3,4],
    ...                      [2,3.25,3.75,4.5,6,7],
    ...                      [2.625,3.625,6.625,9]],
    ...                      key=int))
    [1, 2, 2, 2.625, 3, 3.25, 3.75, 3.625, 4, 4.5, 6, 6.625, 7, 9]


    >>> print list(mergesort([[4, 3, 2, 1],
    ...                      [7, 6, 4.5, 3.75, 3.25, 2],
    ...                      [9, 6.625, 3.625, 2.625]],
    ...                      key=lambda x: -x))
    [9, 7, 6.625, 6, 4.5, 4, 3.75, 3.625, 3.25, 3, 2.625, 2, 2, 1]
    """

    heap = []
    for i, itr in enumerate(iter(pl) for pl in list_of_lists):
        try:
            item = itr.next()
            if key:
                toadd = (key(item), i, item, itr)
            else:
                toadd = (item, i, itr)
            heap.append(toadd)
        except StopIteration:
            pass
    heapq.heapify(heap)

    if key:
        while heap:
            _, idx, item, itr = heap[0]
            yield item
            try:
                item = itr.next()
                heapq.heapreplace(heap, (key(item), idx, item, itr) )
            except StopIteration:
                heapq.heappop(heap)

    else:
        while heap:
            item, idx, itr = heap[0]
            yield item
            try:
                heapq.heapreplace(heap, (itr.next(), idx, itr))
            except StopIteration:
                heapq.heappop(heap)


def remote_iterator(view,name):
    """Return an iterator on an object living on a remote engine.
    """
    view.execute('it%s=iter(%s)'%(name,name), block=True)
    while True:
        try:
            result = view.apply_sync(lambda x: x.next(), Reference('it'+name))
        # This causes the StopIteration exception to be raised.
        except:
            raise e
        else:
            yield result

# Main, interactive testing
if __name__ == '__main__':

    from IPython.parallel import Client, Reference
    rc = Client()
    view = rc[:]
    print('Engine IDs:', rc.ids)

    # Make a set of 'sorted datasets'
    a0 = range(5,20)
    a1 = range(10)
    a2 = range(15,25)

    # Now, imagine these had been created in the remote engines by some long
    # computation.  In this simple example, we just send them over into the
    # remote engines.  They will all be called 'a' in each engine.
    rc[0]['a'] = a0
    rc[1]['a'] = a1
    rc[2]['a'] = a2

    # And we now make a local object which represents the remote iterator
    ra0 = remote_iterator(rc[0],'a')
    ra1 = remote_iterator(rc[1],'a')
    ra2 = remote_iterator(rc[2],'a')

    # Let's merge them, both locally and remotely:
    print('Merge the local datasets:')
    print(list(mergesort([a0,a1,a2])))
    
    print('Locally merge the remote sets:')
    print(list(mergesort([ra0,ra1,ra2])))


Engine IDs: [0, 1, 2, 3]
Merge the local datasets:


C:\Program Files\Anaconda3\lib\site-packages\IPython\parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)


AttributeError: 'range_iterator' object has no attribute 'next'

In [10]:
import ipyparallel as parallel
clients = parallel.Client()
clients.block = True
print(clients.ids)

[0, 1, 2]


In [28]:
clients

In [14]:
def mul(a,b):
    return a*b


NameError: name 'summary' is not defined

In [15]:
mul(5,6)

30

In [19]:
clients[0].apply(mul,5,6)

30

In [21]:
clients[:].apply(mul,5,6)

[30, 30, 30]

In [24]:
view = clients.load_balanced_view()
view.map(mul ,range(1,10), range(2,11))

[2, 6, 12, 20, 30, 42, 56, 72, 90]

1
2
3
4
5
6
7
8
9
